In [1]:
# Install dependencies
!pip install -q datasets transformers[torch] evaluate jiwer torchvision opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.2 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [2]:
!pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 83.8 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [4]:
#%%script echo skipping
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [5]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from datasets import load_dataset, DatasetDict, concatenate_datasets
from transformers import (
    TrOCRProcessor,
    VisionEncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    EarlyStoppingCallback
)
import editdistance  # Required for CER computation


2025-04-15 03:52:57.180360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744689177.393324      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744689177.461378      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# Kaggle-specific configuration
os.environ["TOKENIZERS_PARALLELISM"] = "false"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_GPUS = torch.cuda.device_count()


In [8]:
# Training parameters
MODEL_NAME = "microsoft/trocr-base-handwritten"
MAX_EPOCHS = 10
LEARNING_RATE = 5e-5
IMAGE_SIZE = (384, 384)
BATCH_SIZE = 8 if "T4" in torch.cuda.get_device_name(0) else 4
BATCH_SIZE *= NUM_GPUS
GRADIENT_ACCUMULATION_STEPS = 2 if NUM_GPUS == 1 else 1

In [ ]:
# Training parameters
MODEL_NAME = "microsoft/trocr-base-handwritten"
MAX_EPOCHS = 4
LEARNING_RATE = 5e-5
IMAGE_SIZE = (384, 384)
BATCH_SIZE = 8 if "T4" in torch.cuda.get_device_name(0) else 4
BATCH_SIZE *= NUM_GPUS
GRADIENT_ACCUMULATION_STEPS = 2 if NUM_GPUS == 1 else 1

In [9]:
# Load datasets
iam_dataset = load_dataset("Teklia/IAM-line")
imgur_dataset = load_dataset("staghado/IMGUR-dataset")

# Combine training datasets
combined_train_dataset = concatenate_datasets([iam_dataset["train"], imgur_dataset["train"]])
train_val_split = combined_train_dataset.train_test_split(test_size=0.1, seed=42)


README.md:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/73.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6482 [00:00<?, ? examples/s]

...

chunk_8.zip:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

chunk_9.zip:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

dataset_info.zip:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

imgur8k-dataset.zip:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
# Define OCR Preprocessor
class OCRPreprocessor:
    def __init__(self, model_name, image_size=(224, 224)):
        self.processor = TrOCRProcessor.from_pretrained(model_name)
        self.image_size = image_size

    def _process_image(self, img):
        if img.mode != "L":
            img = img.convert("L")
        img = np.array(img)
        img = cv2.fastNlMeansDenoising(img, h=10)
        img = cv2.medianBlur(img, 3)
        img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        img = cv2.resize(img, self.image_size)
        return img

    def process_batch(self, examples):
        processed_images = []
        for img_data in examples["image"]:
            img = Image.open(img_data) if isinstance(img_data, str) else img_data
            processed_img = Image.fromarray(self._process_image(img)).convert("RGB")
            processed_images.append(processed_img)

        texts = [str(text) if text is not None else "" for text in examples["text"]]
        pixel_values = self.processor(processed_images, return_tensors="pt").pixel_values
        labels = self.processor.tokenizer(
            texts, padding="max_length", max_length=64, return_tensors="pt"
        ).input_ids

        return {"pixel_values": pixel_values, "labels": labels}


In [12]:
# Preprocess datasets with a subset (limit to 400 examples)
preprocessor = OCRPreprocessor(model_name=MODEL_NAME, image_size=IMAGE_SIZE)

# Take only the first 400 examples for training
limited_train_dataset = train_val_split["train"].select(range(400))

# Preprocess the limited dataset
train_dataset = limited_train_dataset.map(
    preprocessor.process_batch,
    batched=True,
    batch_size=16,
    remove_columns=["image", "text"]
)

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [13]:
print(train_dataset)


Dataset({
    features: ['pixel_values', 'labels'],
    num_rows: 400
})


In [14]:
# Take only the first 100 examples for validation
validation_subset = train_val_split["test"].select(range(10))

# Preprocess the validation dataset
validation_dataset = validation_subset.map(
    preprocessor.process_batch,
    batched=True,
    batch_size=16,
    remove_columns=["image", "text"]
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [15]:
# Load and configure the model
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)
model.config.decoder_start_token_id = preprocessor.processor.tokenizer.cls_token_id
model.config.pad_token_id = preprocessor.processor.tokenizer.pad_token_id
model.to(DEVICE)

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (i

In [16]:
# Define CER computation
def calculate_cer(references, predictions):
    total_chars = sum(len(ref) for ref in references)
    total_errors = sum(editdistance.eval(ref, pred) for ref, pred in zip(references, predictions))
    return total_errors / total_chars if total_chars > 0 else 1.0

# Define WER computation
def calculate_wer(references, predictions):
    total_words = sum(len(ref.split()) for ref in references)
    total_errors = sum(editdistance.eval(ref.split(), pred.split()) for ref, pred in zip(references, predictions))
    return total_errors / total_words if total_words > 0 else 1.0





In [22]:
# Corrected training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=MAX_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE // NUM_GPUS if NUM_GPUS > 1 else BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE // NUM_GPUS if NUM_GPUS > 1 else BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    logging_steps=100,
    predict_with_generate=True,
    generation_max_length=64,
    metric_for_best_model="cer",
    load_best_model_at_end=True,
    greater_is_better=False,
    report_to="none",
    warmup_ratio=0.1,
    weight_decay=0.01,
)

In [24]:
from transformers import TrOCRProcessor, Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback

# Load the processor
processor = TrOCRProcessor.from_pretrained(MODEL_NAME)

# Define the compute_metrics function
def compute_metrics(pred):
    predictions = pred.predictions
    labels = pred.label_ids

    # Decode predictions and labels
    decoded_preds = processor.tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Filter valid pairs (ensure no empty labels)
    valid_pairs = [
        (pred.strip(), label.strip())
        for pred, label in zip(decoded_preds, decoded_labels)
        if label.strip()
    ]
    if not valid_pairs:  # If all references are empty
        return {"cer": 1.0, "wer": 1.0}

    decoded_preds, decoded_labels = zip(*valid_pairs)

    # Compute metrics
    cer = calculate_cer(decoded_labels, decoded_preds)  # Implement this
    wer = calculate_wer(decoded_labels, decoded_preds)  # Implement this
    return {"cer": cer, "wer": wer}

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    data_collator=default_data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Cer,Wer
1,No log,0.000000,1.000000,1.000000
2,No log,0.000002,1.000000,1.000000
3,No log,0.000026,1.000000,1.000000
4,0.048700,0.000002,1.000000,1.000000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['decoder.output_projection.weight'].


TrainOutput(global_step=100, training_loss=0.04868200778961182, metrics={'train_runtime': 600.9212, 'train_samples_per_second': 6.656, 'train_steps_per_second': 0.416, 'total_flos': 1.1972563055935488e+18, 'train_loss': 0.04868200778961182, 'epoch': 4.0})

In [31]:
# Subset IAM test dataset for evaluation (limit to 100 examples)
subset_test_dataset = iam_dataset["test"].select(range(300))  # Take only the first 100 examples

# Preprocess the test dataset
processed_test_dataset = subset_test_dataset.map(
    preprocessor.process_batch,
    batched=True,
    batch_size=16,
    remove_columns=["image", "text"]
)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [32]:
# Evaluate the model on the IAM test subset
results = trainer.evaluate(processed_test_dataset, metric_key_prefix="test")
print(f"Test CER (IAM dataset subset): {results['test_cer']*100:.2f}%")
print(f"Test WER (IAM dataset subset): {results['test_wer']*100:.2f}%")

early stopping required metric_for_best_model, but did not find eval_cer so early stopping is disabled


Test CER (IAM dataset subset): 100.00%
Test WER (IAM dataset subset): 100.00%


In [ ]:
# Check if the target CER and WER are met
if results['test_cer'] * 100 <= 7 and results['test_wer'] * 100 <= 15:
    print("Target achieved: CER ≤ 7% and WER ≤ 15%")
else:
    print("Target not achieved. Adjust training or preprocessing.")

In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from datasets import load_dataset, DatasetDict, concatenate_datasets
from transformers import (
    TrOCRProcessor,
    VisionEncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    EarlyStoppingCallback
)
import editdistance  # Required for CER computation

# Kaggle-specific configuration
os.environ["TOKENIZERS_PARALLELISM"] = "false"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_GPUS = torch.cuda.device_count()

# Training parameters
MODEL_NAME = "microsoft/trocr-base-handwritten"
MAX_EPOCHS = 10
LEARNING_RATE = 5e-5
IMAGE_SIZE = (384, 384)
BATCH_SIZE = 8 if "T4" in torch.cuda.get_device_name(0) else 4
BATCH_SIZE *= NUM_GPUS
GRADIENT_ACCUMULATION_STEPS = 2 if NUM_GPUS == 1 else 1

# Define CER computation
def calculate_cer(references, predictions):
    total_chars = sum(len(ref) for ref in references)
    total_errors = sum(editdistance.eval(ref, pred) for ref, pred in zip(references, predictions))
    return total_errors / total_chars if total_chars > 0 else 1.0

# Define WER computation
def calculate_wer(references, predictions):
    total_words = sum(len(ref.split()) for ref in references)
    total_errors = sum(editdistance.eval(ref.split(), pred.split()) for ref, pred in zip(references, predictions))
    return total_errors / total_words if total_words > 0 else 1.0

# Define OCR Preprocessor
class OCRPreprocessor:
    def __init__(self, model_name, image_size=(224, 224)):
        self.processor = TrOCRProcessor.from_pretrained(model_name)
        self.image_size = image_size

    def _process_image(self, img):
        if img.mode != "L":
            img = img.convert("L")
        img = np.array(img)
        img = cv2.fastNlMeansDenoising(img, h=10)
        img = cv2.medianBlur(img, 3)
        img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        img = cv2.resize(img, self.image_size)
        return img

    def process_batch(self, examples):
        processed_images = []
        for img_data in examples["image"]:
            img = Image.open(img_data) if isinstance(img_data, str) else img_data
            processed_img = Image.fromarray(self._process_image(img)).convert("RGB")
            processed_images.append(processed_img)

        texts = [str(text) if text is not None else "" for text in examples["text"]]
        pixel_values = self.processor(processed_images, return_tensors="pt").pixel_values
        labels = self.processor.tokenizer(
            texts, padding="max_length", max_length=64, return_tensors="pt"
        ).input_ids

        return {"pixel_values": pixel_values, "labels": labels}

# Load datasets
iam_dataset = load_dataset("Teklia/IAM-line")
imgur_dataset = load_dataset("staghado/IMGUR-dataset")

# Combine training datasets
combined_train_dataset = concatenate_datasets([iam_dataset["train"], imgur_dataset["train"]])
train_val_split = combined_train_dataset.train_test_split(test_size=0.1, seed=42)

# Preprocess datasets
preprocessor = OCRPreprocessor(model_name=MODEL_NAME, image_size=IMAGE_SIZE)
train_dataset = train_val_split["train"].map(
    preprocessor.process_batch,
    batched=True,
    batch_size=16,
    remove_columns=["image", "text"]
)
validation_dataset = train_val_split["test"].map(
    preprocessor.process_batch,
    batched=True,
    batch_size=16,
    remove_columns=["image", "text"]
)

# Subset IAM test dataset for evaluation
subset_test_dataset = iam_dataset["test"].select(range(500))
processed_test_dataset = subset_test_dataset.map(
    preprocessor.process_batch,
    batched=True,
    batch_size=16,
    remove_columns=["image", "text"]
)

# Load and configure the model
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)
model.config.decoder_start_token_id = preprocessor.processor.tokenizer.cls_token_id
model.config.pad_token_id = preprocessor.processor.tokenizer.pad_token_id
model.to(DEVICE)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=MAX_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE // NUM_GPUS if NUM_GPUS > 1 else BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE // NUM_GPUS if NUM_GPUS > 1 else BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    predict_with_generate=True,
    generation_max_length=64,
    metric_for_best_model="cer",
    load_best_model_at_end=True,
    greater_is_better=False,
    report_to="none",
    warmup_ratio=0.1,
    weight_decay=0.01,
)

# Define compute metrics
def compute_metrics(pred):
    predictions = pred.predictions
    labels = pred.label_ids

    decoded_preds = preprocessor.processor.tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = preprocessor.processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = zip(
        *[(pred.strip(), label.strip()) for pred, label in zip(decoded_preds, decoded_labels) if label.strip()]
    ) if decoded_labels else ([], [])

    cer = calculate_cer(decoded_labels, decoded_preds)
    wer = calculate_wer(decoded_labels, decoded_preds)

    return {"cer": cer, "wer": wer}

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    data_collator=default_data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train the model
trainer.train()

# Evaluate the model on the IAM test subset
results = trainer.evaluate(processed_test_dataset, metric_key_prefix="test")
print(f"Test CER (IAM dataset subset): {results['test_cer']*100:.2f}%")
print(f"Test WER (IAM dataset subset): {results['test_wer']*100:.2f}%")

# Check if the target CER and WER are met
if results['test_cer'] * 100 <= 7 and results['test_wer'] * 100 <= 15:
    print("Target achieved: CER ≤ 7% and WER ≤ 15%")
else:
    print("Target not achieved. Adjust training or preprocessing.")